## Preparation

In [1]:
import bigframes
import bigframes.pandas as bpd

Enable the semantic operator experiment

In [2]:
bigframes.options.experiments.semantic_operators = True

/usr/local/google/home/sycai/src/python-bigquery-dataframes/bigframes/_config/experiment_options.py:33: UserWarning: Semantic operators are still under experiments, and are subject to change in the future.
  warnings.warn(


Prepare the LLM model. Here we are going to use Gemini 1.5 Flash.

In [3]:
import bigframes.ml.llm as llm
model = llm.GeminiTextGenerator(model_name=llm._GEMINI_1P5_FLASH_001_ENDPOINT)

/usr/local/google/home/sycai/src/python-bigquery-dataframes/bigframes/pandas/__init__.py:559: DefaultLocationWarning: No explicit location is set, so using location US for the session.
  return global_session.get_global_session()


## Semantic Filtering

In [4]:
df = bpd.DataFrame({'country': ['USA', 'Germany'], 'city': ['Seattle', 'Berlin']})
df.semantics.filter("{city} is the capital of {country}", model)

/usr/local/google/home/sycai/src/python-bigquery-dataframes/bigframes/core/__init__.py:112: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(


,country,city
1,Germany,Berlin


## Semantic Mapping

In [5]:
df = bpd.DataFrame(
        data={"ingredient_1": ["Burger Bun", "Soy Bean"], "ingredient_2": ["Beef Patty", "Bittern"]}
    )

In [6]:
df.semantics.map("What is the food made from {ingredient_1} and {ingredient_2}? One word only.", result_column_name="food", model=model)

/usr/local/google/home/sycai/src/python-bigquery-dataframes/bigframes/core/__init__.py:112: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(


,ingredient_1,ingredient_2,food
0,Burger Bun,Beef Patty,Burger
1,Soy Bean,Bittern,Tofu
